In [1]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token:str, **kwargs) -> None:
        print(f"Received streaming token {token}, kwargs:{kwargs}")
        
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Received non-streaming response: {response}")

In [2]:
# from typing import List
# from langchain_core.language_models.chat_models import BaseChatModel
# from langchain.schema.messages import BaseMessage, AIMessage, HumanMessage
# from langchain.schema.output import ChatResult, ChatGeneration
# from langchain_core.callbacks import Callbacks
# from llmstudio.llm import LLM
# from typing import Any, Optional
# from langchain.chat_models import ChatOpenAI
# from pydantic import BaseModel, Field

# class ChatLLMstudio(ChatOpenAI):
#     model_id: str
#     llm: LLM = Field(default=None)


#     def __init__(self, model_id: str, **kwargs):
#         super().__init__(model_id=model_id, **kwargs)
#         self.llm = LLM(model_id, **kwargs)
    
#     @property
#     def _llm_type(self):
#         return "LLMstudio"

#     def _generate(self, messages: List[BaseMessage], **kwargs) -> ChatResult:
#         messages_dicts, _ = self._create_message_dicts(messages, [])
#         response = self.llm.chat(messages_dicts, **kwargs)
#         return self._create_chat_result(response)

In [3]:
from llmstudio.llm.langchain import ChatLLMstudio

In [4]:
gpt3 = ChatLLMstudio(model_id='azure/"GPT-4-32k-playground"', api_endpoint="https://mdclonedev-openaisvc-cn.openai.azure.com/", api_version="2023-07-01-preview", temperature=0)

Running Engine on localhost:8000Running Tracking on http://localhost:8080



In [5]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def cancel_ticket(ticket_number: str):
    """Use this to cancel a ticket"""
    return "Ticket cancelled"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

@tool
def make_complaint(complaint: str):
    """Use this to forward a complaint to the complaint department"""
    return "Complaint forwarded. We appreciate your feedback."


def assistant(question: str)->str:
    """YOUR CODE HERE"""
    tools = [get_departure, cancel_ticket, buy_ticket, make_complaint]

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, gpt3, agent=AgentType.OPENAI_MULTI_FUNCTIONS, handle_parsing_errors="Check your output and make sure it conforms!", verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

INFO:     Started server process [27653]
INFO:     Waiting for application startup.
INFO:     Started server process [27653]
INFO:     Application startup complete.
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://localhost:8080 (Press CTRL+C to quit)


In [6]:
assistant('When does my train depart? My ticket number is 1234')




> Entering new AgentExecutor chain...
INFO:     ::1:62329 - "POST /api/engine/chat/azure HTTP/1.1" 400 Bad Request


HTTPError: 400 Client Error: Bad Request for url: http://localhost:8000/api/engine/chat/azure